In [5]:
#importing libraries
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import bs4 as BeautifulSoup
import urllib.request 
import requests

In [9]:
#fetching the content from the URL
fetched_data = urllib.request.urlopen('https://www.wsj.com/')

In [22]:
import requests
page = requests.get("https://www.wsj.com/")
page.content

b'<!DOCTYPE html><html data-region="na,us" data-protocol="https" data-reactid=".1r0025c9ng8" data-react-checksum="1626410956"><head data-reactid=".1r0025c9ng8.0"><title data-reactid=".1r0025c9ng8.0.0">The Wall Street Journal &amp; Breaking News, Business, Financial and Economic News, World News and Video</title><meta http-equiv="X-UA-Compatible" content="IE=edge" data-reactid=".1r0025c9ng8.0.1"/><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" data-reactid=".1r0025c9ng8.0.2"/><meta name="viewport" content="initial-scale=1.0001, minimum-scale=1.0001,\n        maximum-scale=1.0001, user-scalable=no" data-reactid=".1r0025c9ng8.0.3"/><meta name="description" content="WSJ online coverage of breaking news and current headlines from the US and around the world. Top stories, photos, videos, detailed analysis and in-depth reporting." data-reactid=".1r0025c9ng8.0.4"/><meta name="keywords" content="News, breaking news, latest news, US news, headlines, world news, business, finan

In [32]:
article_read = fetched_data.read()

In [33]:
#parsing the URL content and storing in a variable
article_parsed = BeautifulSoup.BeautifulSoup(article_read,'html.parser')

In [34]:
#returning <p> tags
paragraphs = article_parsed.find_all('p')

In [35]:
article_content = ''

#looping through the paragraphs and adding them to the variable
for p in paragraphs:  
    article_content += p.text

In [36]:
def _create_dictionary_table(text_string) -> dict:
   
    #removing stop words
    stop_words = set(stopwords.words("english"))
    
    words = word_tokenize(text_string)
    
    #reducing words to their root form
    stem = PorterStemmer()
    
    #creating dictionary for the word frequency table
    frequency_table = dict()
    for wd in words:
        wd = stem.stem(wd)
        if wd in stop_words:
            continue
        if wd in frequency_table:
            frequency_table[wd] += 1
        else:
            frequency_table[wd] = 1

    return frequency_table

In [37]:
def _calculate_sentence_scores(sentences, frequency_table) -> dict:   

    #algorithm for scoring a sentence by its words
    sentence_weight = dict()

    for sentence in sentences:
        sentence_wordcount = (len(word_tokenize(sentence)))
        sentence_wordcount_without_stop_words = 0
        for word_weight in frequency_table:
            if word_weight in sentence.lower():
                sentence_wordcount_without_stop_words += 1
                if sentence[:7] in sentence_weight:
                    sentence_weight[sentence[:7]] += frequency_table[word_weight]
                else:
                    sentence_weight[sentence[:7]] = frequency_table[word_weight]

        sentence_weight[sentence[:7]] = sentence_weight[sentence[:7]] / sentence_wordcount_without_stop_words

       

    return sentence_weight

In [38]:
def _calculate_average_score(sentence_weight) -> int:
   
    #calculating the average score for the sentences
    sum_values = 0
    for entry in sentence_weight:
        sum_values += sentence_weight[entry]

    #getting sentence average value from source text
    average_score = (sum_values / len(sentence_weight))

    return average_score

In [39]:
def _get_article_summary(sentences, sentence_weight, threshold):
    sentence_counter = 0
    article_summary = ''

    for sentence in sentences:
        if sentence[:7] in sentence_weight and sentence_weight[sentence[:7]] >= (threshold):
            article_summary += " " + sentence
            sentence_counter += 1

    return article_summary

In [40]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nancy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nancy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [43]:
def _run_article_summary(article):
    
    #creating a dictionary for the word frequency table
    frequency_table = _create_dictionary_table(article)

    #tokenizing the sentences
    sentences = sent_tokenize(article)

    #algorithm for scoring a sentence by its words
    sentence_scores = _calculate_sentence_scores(sentences, frequency_table)

    #getting the threshold
    threshold = _calculate_average_score(sentence_scores)

    #producing the summary
    article_summary = _get_article_summary(sentences, sentence_scores, 1 * threshold)

    return article_summary

if __name__ == '__main__':
    summary_results = _run_article_summary(article_content)
    print(summary_results)

 “It’s not a membership-based organization. But unlike Al Qaeda then, the Islamic State already has numerous affiliates around the globe, an influential media ministry and thousands of fighters still underground in the group’s home base in Iraq and Syria.As early as 2015, ISIS began instructing recruits to migrate to territory held by its overseas affiliates. But the next week, Naba came out on schedule.The Islamic State can still tap a large war chest to finance its global operations. Stealthily distributing the money abroad is a skill that the group has developed over the years, using proxies, cutouts and known middlemen, he said.Decentralization makes it difficult to know the extent of the group’s involvement in attacks like the one in Sri Lanka.The Islamic State has claimed responsibility for the bombings, but experts and the local authorities have yet to determine the extent of its ties to the local group said to have carried them out. Until this week, the group, National Thowheet

References
https://blog.floydhub.com/gentle-introduction-to-text-summarization-in-machine-learning/